## Transformations

Transformations are lazy operations on a RDD that create one or many new RDDs.
* Transformations are functions that take a RDD as the input and produce one or many RDDs as the output. They do not change the input RDD (since RDDs are immutable and hence cannot be modified), but always produce one or more new RDDs by applying the computations they represent.
* By applying transformations you incrementally build a RDD lineage with all the parent RDDs of the final RDD(s).
* Transformations are lazy, i.e. are not executed immediately. Only after calling an action are transformations executed.

###  Transformations Types

Spark carefully distinguish "transformation" operation in two types:
![Narrow vs wide transformations Image](assets/narrow_vs_wide_transformations.png)

* **Narrow Transformations ** are the result of map, filter and such that is from the data from a single partition only, i.e. it is self-sustained.
    * An output RDD has partitions with records that originate from a single partition in the parent RDD. Only a limited subset of partitions used to calculate the result.
    * Spark groups narrow transformations as a stage which is called pipelining.
    

* **Wide Transformations ** are the result of groupByKey and reduceByKey. The data required to compute the records in a single partition may reside in many partitions of the parent RDD.
    * All of the tuples with the same key must end up in the same partition, processed by the same task. To satisfy these operations, Spark must execute RDD shuffle, which transfers data across cluster and results in a new stage with a new set of partitions.


###  Transformations dependecies

The figure below gives a quick overview of the flow of a spark job
![spark schedule process Image](assets/spark_schedule-process.png)


One of the challenges in providing RDDs as an abstraction is choosing a representation for them that can 
track lineage across a wide range of transformations. The most interesting question in designing this  interface is how to represent dependencies between RDDs.

It is both sufficient and useful to classify  dependencies into two types: 
* **Narrow dependencies**, where each partition of the parent RDD is used by at most one 
partition  of the child RDD.
* **Wide dependencies**, where multiple child partitions may depend on it.

![Transformation dependencies Image](assets/Transformation_Dependencies.png)


### General Transformations

* **filter** (f)
        
    Return a new RDD containing only the elements that satisfy a predicate.

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [2]:
rdd = sc.parallelize(range(-5,5))          # Rango (-5, 5)
filtered_rdd = rdd.filter(lambda x: x>=0)   # Devuelve los positivos
print(filtered_rdd.collect())

[0, 1, 2, 3, 4]


* **map** (f, preservesPartitioning=False)
    
    Return a new RDD by applying a function to each element of this RDD.

In [3]:
def add1(x):
    return(x+1)

squared_rdd = (filtered_rdd
               .map(add1)
               .map(lambda x: (x, x*x))) 
print(squared_rdd.collect())

[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]


 * **flatMap** (f, preservesPartitioning=False)

    Return a new RDD by first applying a function to all elements of this RDD, and then flattening the results.

In [4]:
squaredflat_rdd = (filtered_rdd
                   .map(add1)
                   .flatMap(lambda x: (x, x*x)))
print(squaredflat_rdd.collect())

[1, 1, 2, 4, 3, 9, 4, 16, 5, 25]


* **distinct**(numPartitions=None)

    Return a new RDD containing the distinct elements in this RDD.

In [5]:
distinct_rdd = squaredflat_rdd.distinct()
print(distinct_rdd.collect())

[2, 4, 16, 1, 3, 9, 5, 25]


* **groupBy**(f, numPartitions=None, partitionFunc=<function portable_hash at 0x7fc35dbc8e60>)

    Return an RDD of grouped items.

In [6]:
# Group values depending on the remainder of the division by 3
grouped_rdd = distinct_rdd.groupBy(lambda x: x%3)
print(grouped_rdd.collect())
print([(x,sorted(y)) for (x,y) in grouped_rdd.collect()])

[(2, <pyspark.resultiterable.ResultIterable object at 0x7f8f50609c40>), (0, <pyspark.resultiterable.ResultIterable object at 0x7f8f505ac130>), (1, <pyspark.resultiterable.ResultIterable object at 0x7f8f505ac070>)]
[(2, [2, 5]), (0, [3, 9]), (1, [1, 4, 16, 25])]


### Math/Statistical Transformations

   
* **sample** (withReplacement, fraction, seed=None)

    Return a sampled subset of this RDD.

    Parameters:	
        * withReplacement – can elements be sampled multiple times (replaced when sampled out)
        * fraction – expected size of the sample as a fraction of this RDD’s size without replacement: probability that each element is chosen; fraction must be [0, 1] with replacement: expected number of times each element is chosen; fraction must be >= 0
        * seed – seed for the random number generator

In [7]:
s1 = squaredflat_rdd.sample(False, 0.5).collect()
s2 = squaredflat_rdd.sample(True, 2).collect()
s3 = squaredflat_rdd.sample(False, 0.8).collect()
print('s1={0}\ns2={1}\ns3={2}'.format(s1, s2, s3))

s1=[1, 3, 4, 25]
s2=[1, 1, 2, 2, 4, 3, 3, 4, 4, 4, 16, 16, 5, 25, 25, 25]
s3=[1, 2, 4, 3, 9, 4, 16, 5]


### Set Theory /Relationan Transformations (transformations on two RDDs)

* **union**(other)

    Return the union of this RDD and another one.

In [8]:
rdda = sc.parallelize(['a', 'b', 'c'])
rddb = sc.parallelize(['c', 'd', 'e'])
rddu = rdda.union(rddb)
print(rddu.collect())

['a', 'b', 'c', 'c', 'd', 'e']


* **intersection** (other)

    Return the intersection of this RDD and another one. The output will not contain any duplicate elements, even if the input RDDs did.
    * This method performs a shuffle internally. 

In [9]:
rddi = rdda.intersection(rddb)
print(rddi.collect())

['c']


* **subtract** (other, numPartitions=None)

    Return each value in self that is not contained in other.

In [10]:
rdds = rdda.subtract(rddb)
print(rdds.collect())

['b', 'a']


* **zip** (other)

    Zips this RDD with another one, returning key-value pairs with the first element in each RDD second element in each RDD, etc. 
    * Assumes that the two RDDs have the same number of partitions and the same number of elements in each partition (e.g. one was made through a map on the other).

In [11]:
x = sc.parallelize(range(0,5))
y = sc.parallelize(range(1000, 1005))
x.zip(y).collect()

[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]